# Institut des Algorithmes du Sénégal

# Tuto 1

In [ ]:
# code by Institut des Algorithmes du Sénégal
import torch
import torch.nn as nn
import torch.optim as optim

def make_batch():
    input_batch = []
    target_batch = []

    for sen in sentences:
        word = sen.split() # space tokenizer
        input = [word_dict[n] for n in word[:-1]] # create (1~n-1) as input
        target = word_dict[word[-1]] # create (n) as target, We usually call this 'casual language model'

        input_batch.append(input)
        target_batch.append(target)

    return input_batch, target_batch

# Model
class NNLM(nn.Module):
    def __init__(self):
        super(NNLM, self).__init__()
        self.C = nn.Embedding(n_class, m)
        self.H = nn.Linear(n_step * m, n_hidden, bias=False)
        self.d = nn.Parameter(torch.ones(n_hidden))
        self.U = nn.Linear(n_hidden, n_class, bias=False)
        self.W = nn.Linear(n_step * m, n_class, bias=False)
        self.b = nn.Parameter(torch.ones(n_class))

    def forward(self, X):
        X = self.C(X) # X : [batch_size, n_step, m]
        X = X.view(-1, n_step * m) # [batch_size, n_step * m]
        tanh = torch.tanh(self.d + self.H(X)) # [batch_size, n_hidden]
        output = self.b + self.W(X) + self.U(tanh) # [batch_size, n_class]
        return output

if __name__ == '__main__':
    n_step = 2 # number of steps, n-1 in paper
    n_hidden = 2 # number of hidden size, h in paper
    m = 2 # embedding size, m in paper

    sentences = ["i like dog", "i love coffee", "i hate milk"]

    word_list = " ".join(sentences).split()
    word_list = list(set(word_list))
    word_dict = {w: i for i, w in enumerate(word_list)}
    number_dict = {i: w for i, w in enumerate(word_list)}
    n_class = len(word_dict)  # number of Vocabulary

    model = NNLM()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    input_batch, target_batch = make_batch()
    input_batch = torch.LongTensor(input_batch)
    target_batch = torch.LongTensor(target_batch)

    # Training
    for epoch in range(5000):
        optimizer.zero_grad()
        output = model(input_batch)

        # output : [batch_size, n_class], target_batch : [batch_size]
        loss = criterion(output, target_batch)
        if (epoch + 1) % 1000 == 0:
            print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

        loss.backward()
        optimizer.step()

    # Predict
    predict = model(input_batch).data.max(1, keepdim=True)[1]

    # Test
    print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])

Epoch: 1000 cost = 0.293031
Epoch: 2000 cost = 0.026870
Epoch: 3000 cost = 0.007537
Epoch: 4000 cost = 0.003010
Epoch: 5000 cost = 0.001401
[['i', 'like'], ['i', 'love'], ['i', 'hate']] -> ['dog', 'coffee', 'milk']


# Tuto 2

In [8]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from autocorrect import spell
from nltk.wsd import lesk
from nltk.tokenize import sent_tokenize
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
text = "In this book authored by Sohom Ghosh and Dwight Gunning, we shall learnning how to pracess Natueral Language and extract insights from it. The first four chapter will introduce you to the basics of NLP. Later chapters will describe how to deal with complex NLP prajects. If you want to get early access of it, you should book your order now."

In [10]:
words = word_tokenize(text)
print(words[0:20])

['In', 'this', 'book', 'authored', 'by', 'Sohom', 'Ghosh', 'and', 'Dwight', 'Gunning', ',', 'we', 'shall', 'learnning', 'how', 'to', 'pracess', 'Natueral', 'Language', 'and']


In [12]:
corrected_sentence = ""
corrected_word_list = []
for wd in words:
    if wd not in string.punctuation:
        wd_c = spell(wd)
        if wd_c != wd:
            print(wd+" has been corrected to: "+wd_c)
            corrected_sentence = corrected_sentence+" "+wd_c
            corrected_word_list.append(wd_c)
        else:
            corrected_sentence = corrected_sentence+" "+wd
            corrected_word_list.append(wd)
    else:
        corrected_sentence = corrected_sentence + wd
        corrected_word_list.append(wd)



autocorrect.spell is deprecated,             use autocorrect.Speller instead
autocorrect.spell is deprecated,             use autocorrect.Speller instead
autocorrect.spell is deprecated,             use autocorrect.Speller instead
autocorrect.spell is deprecated,             use autocorrect.Speller instead
autocorrect.spell is deprecated,             use autocorrect.Speller instead
autocorrect.spell is deprecated,             use autocorrect.Speller instead
Sohom has been corrected to: Show
autocorrect.spell is deprecated,             use autocorrect.Speller instead
Ghosh has been corrected to: Ghost
autocorrect.spell is deprecated,             use autocorrect.Speller instead
autocorrect.spell is deprecated,             use autocorrect.Speller instead
Dwight has been corrected to: Right
autocorrect.spell is deprecated,             use autocorrect.Speller instead
Gunning has been corrected to: Running
autocorrect.spell is deprecated,             use autocorrect.Speller instead
autocorre

In [13]:
corrected_sentence

' In this book authored by Show Ghost and Right Running, we shall learning how to process Natural Language and extract insights from it. The first four chapter will introduce you to the basics of LP. Later chapters will describe how to deal with complex LP projects. If you want to get early access of it, you should book your order now.'

In [14]:
print(corrected_word_list[0:20])

['In', 'this', 'book', 'authored', 'by', 'Show', 'Ghost', 'and', 'Right', 'Running', ',', 'we', 'shall', 'learning', 'how', 'to', 'process', 'Natural', 'Language', 'and']


In [15]:
print(nltk.pos_tag(corrected_word_list))

[('In', 'IN'), ('this', 'DT'), ('book', 'NN'), ('authored', 'VBN'), ('by', 'IN'), ('Show', 'NNP'), ('Ghost', 'NNP'), ('and', 'CC'), ('Right', 'NNP'), ('Running', 'NNP'), (',', ','), ('we', 'PRP'), ('shall', 'MD'), ('learning', 'VB'), ('how', 'WRB'), ('to', 'TO'), ('process', 'VB'), ('Natural', 'NNP'), ('Language', 'NNP'), ('and', 'CC'), ('extract', 'JJ'), ('insights', 'NNS'), ('from', 'IN'), ('it', 'PRP'), ('.', '.'), ('The', 'DT'), ('first', 'JJ'), ('four', 'CD'), ('chapter', 'NN'), ('will', 'MD'), ('introduce', 'VB'), ('you', 'PRP'), ('to', 'TO'), ('the', 'DT'), ('basics', 'NNS'), ('of', 'IN'), ('LP', 'NNP'), ('.', '.'), ('Later', 'NNP'), ('chapters', 'NNS'), ('will', 'MD'), ('describe', 'VB'), ('how', 'WRB'), ('to', 'TO'), ('deal', 'VB'), ('with', 'IN'), ('complex', 'JJ'), ('LP', 'NNP'), ('projects', 'NNS'), ('.', '.'), ('If', 'IN'), ('you', 'PRP'), ('want', 'VBP'), ('to', 'TO'), ('get', 'VB'), ('early', 'JJ'), ('access', 'NN'), ('of', 'IN'), ('it', 'PRP'), (',', ','), ('you', 'PRP'

In [18]:
# Downloads the data.
import nltk
nltk.download('stopwords')
# Using the stopwords.
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
corrected_word_list_without_stopwords = []
for wd in corrected_word_list:
    if wd not in stop_words:
        corrected_word_list_without_stopwords.append(wd)
corrected_word_list_without_stopwords[:20]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['In',
 'book',
 'authored',
 'Show',
 'Ghost',
 'Right',
 'Running',
 ',',
 'shall',
 'learning',
 'process',
 'Natural',
 'Language',
 'extract',
 'insights',
 '.',
 'The',
 'first',
 'four',
 'chapter']

In [19]:
stemmer = nltk.stem.PorterStemmer()
corrected_word_list_without_stopwords_stemmed = []
for wd in corrected_word_list_without_stopwords:
    corrected_word_list_without_stopwords_stemmed.append(stemmer.stem(wd))
corrected_word_list_without_stopwords_stemmed[:20]

['In',
 'book',
 'author',
 'show',
 'ghost',
 'right',
 'run',
 ',',
 'shall',
 'learn',
 'process',
 'natur',
 'languag',
 'extract',
 'insight',
 '.',
 'the',
 'first',
 'four',
 'chapter']

In [20]:
lemmatizer = WordNetLemmatizer()
corrected_word_list_without_stopwords_lemmatized = []
for wd in corrected_word_list_without_stopwords:
    corrected_word_list_without_stopwords_lemmatized.append(lemmatizer.lemmatize(wd))
corrected_word_list_without_stopwords_lemmatized[:20]

['In',
 'book',
 'authored',
 'Show',
 'Ghost',
 'Right',
 'Running',
 ',',
 'shall',
 'learning',
 'process',
 'Natural',
 'Language',
 'extract',
 'insight',
 '.',
 'The',
 'first',
 'four',
 'chapter']

In [21]:
print(sent_tokenize(corrected_sentence))

[' In this book authored by Show Ghost and Right Running, we shall learning how to process Natural Language and extract insights from it.', 'The first four chapter will introduce you to the basics of LP.', 'Later chapters will describe how to deal with complex LP projects.', 'If you want to get early access of it, you should book your order now.']
